# **BentoML Example: Linear Regression with Paddlepaddle**
**BentoML makes moving trained ML models to production easy:**



*   Package models trained with any ML framework and reproduce them for model serving in production
* **Deploy anywhere** for online API serving or offline batch serving
* High-Performance API model server with adaptive micro-batching support
* Central hub for managing models and deployment process via Web UI and APIs
* Modular and flexible design making it adaptable to your infrastrcuture

BentoML is a framework for serving, managing, and deploying machine learning models. It is aiming to bridge the gap between Data Science and DevOps, and enable teams to deliver prediction services in a fast, repeatable, and scalable way.

Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML.

This notebook demonstrates how to use BentoML to turn a paddlepaddle model into a docker image containing a REST API server serving this model, how to use your ML service built with BentoML as a CLI tool, and how to distribute it a pypi package.

The example is based on [this tutorial](https://www.paddlepaddle.org.cn/documentation/docs/en/1.5/beginners_guide/basics/fit_a_line/README.html), using dataset from the [UCI Machine Learning Repository](https://www.kaggle.com/schirmerchad/bostonhoustingmlnd)

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
!pip install bentoml

#!pip install -q bentoml 'paddlepaddle>=2.0.0' 'pandas>=1.1.1' 'numpy>=1.8.2'
!pip install paddlepaddle

  Cloning https://github.com/bentoml/BentoML.git to /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/pip-req-build-y5l3v5a6
  Running command git clone -q https://github.com/bentoml/BentoML.git /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/pip-req-build-y5l3v5a6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


  Created wheel for BentoML: filename=BentoML-0.12.0+15.g7d40998-py3-none-any.whl size=674023 sha256=5bd65f7f3eadd7e78aad7142a77ecf2619efe6b6147f1fc8fb0fb7be343a17d5
  Stored in directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/pip-ephem-wheel-cache-taoztxg4/wheels/dd/93/68/1a11f763a858794f96abf3c1def8b101125e236de566ebeb25
Successfully built BentoML
  Attempting uninstall: BentoML
    Found existing installation: BentoML 0.12.0+14.gb6fe25cf
    Uninstalling BentoML-0.12.0+14.gb6fe25cf:
      Successfully uninstalled BentoML-0.12.0+14.gb6fe25cf
     |████████████████████████████████| 74.9 MB 99 kB/s s eta 0:00:01


In [6]:
import numpy as np
import paddle
import paddle.nn as nn
import paddle.optimizer as opt
import pandas as pd

import bentoml

/usr/local/Caskroom/miniconda/base/envs/dev/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# **Prepare Dataset**

In [7]:
import paddle
import numpy as np
import bentoml
from paddle.static import InputSpec

BATCH_SIZE = 8
BATCH_NUM = 4
EPOCH_NUM = 5

IN_FEATURES = 13
OUT_FEATURES = 1

class LinearNet(paddle.nn.Layer):
    def __init__(self):
        super(LinearNet, self).__init__()
        self._linear = paddle.nn.Linear(IN_FEATURES, OUT_FEATURES)

    @paddle.jit.to_static(input_spec=[InputSpec(shape=[IN_FEATURES], dtype='float32')])
    def forward(self, x):
        return self._linear(x)

    def train(self, loader, loss_fn, opt):
        for epoch_id in range(EPOCH_NUM):
            for batch_id, (image, label) in enumerate(loader()):
                out = self._linear(image)
                loss = loss_fn(out, label)
                loss.backward()
                opt.step()
                opt.clear_grad()
                print("Epoch {} batch {}: loss = {}".format(
                    epoch_id, batch_id, np.mean(loss.numpy())))

/usr/local/Caskroom/miniconda/base/envs/dev/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  return (isinstance(seq, collections.Sequence) and


# **Model Training**

In [8]:
model = LinearNet()
loss_fn = paddle.nn.MSELoss()
adam = paddle.optimizer.Adam(parameters=model.parameters())

dataset = paddle.text.datasets.UCIHousing(mode="train")

loader = paddle.io.DataLoader(dataset,
  batch_size=BATCH_SIZE,
  shuffle=True,
  drop_last=True,
  num_workers=2)

model.train(loader, loss_fn, adam)

Cache file /Users/bozhaoyu/.cache/paddle/dataset/uci_housing/housing.data not found, downloading http://paddlemodels.bj.bcebos.com/uci_housing/housing.data 
Begin to download
............
Download finished
/usr/local/Caskroom/miniconda/base/envs/dev/lib/python3.7/site-packages/paddle/fluid/reader.py:352: UserWarning: DataLoader with multi-process mode is not supported on MacOs and Windows currently. Please use signle-process mode with num_workers = 0 instead
  "DataLoader with multi-process mode is not supported on MacOs and Windows currently." \
/usr/local/Caskroom/miniconda/base/envs/dev/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdoc

Epoch 0 batch 0: loss = 435.19635009765625
Epoch 0 batch 1: loss = 421.4579772949219
Epoch 0 batch 2: loss = 459.74871826171875
Epoch 0 batch 3: loss = 680.4847412109375
Epoch 0 batch 4: loss = 858.70458984375
Epoch 0 batch 5: loss = 536.1829833984375
Epoch 0 batch 6: loss = 555.8958740234375
Epoch 0 batch 7: loss = 388.1265563964844
Epoch 0 batch 8: loss = 647.7759399414062
Epoch 0 batch 9: loss = 494.9188232421875
Epoch 0 batch 10: loss = 970.626220703125
Epoch 0 batch 11: loss = 659.16064453125
Epoch 0 batch 12: loss = 983.6190185546875
Epoch 0 batch 13: loss = 936.4552612304688
Epoch 0 batch 14: loss = 1019.9678344726562
Epoch 0 batch 15: loss = 747.4344482421875
Epoch 0 batch 16: loss = 831.9373168945312
Epoch 0 batch 17: loss = 350.56976318359375
Epoch 0 batch 18: loss = 515.817626953125
Epoch 0 batch 19: loss = 713.2019653320312
Epoch 0 batch 20: loss = 610.8594360351562
Epoch 0 batch 21: loss = 594.1142578125
Epoch 0 batch 22: loss = 471.9237060546875
Epoch 0 batch 23: loss = 1

In [9]:
 test_x = np.array([[-0.0405441 ,  0.06636364, -0.32356227, -0.06916996, -0.03435197,
        0.05563625, -0.03475696,  0.02682186, -0.37171335, -0.21419304,
       -0.33569506,  0.10143217, -0.21172912]]).astype('float32')

df_test_x = pd.DataFrame(test_x)

In [10]:
import csv

with open('test.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(df_test_x.columns)
    spamwriter.writerow([-0.0405441 ,  0.06636364, -0.32356227, -0.06916996, -0.03435197,
        0.05563625, -0.03475696,  0.02682186, -0.37171335, -0.21419304,
       -0.33569506,  0.10143217, -0.21172912])

# **Create BentoService for model serving**

In [11]:
%%writefile paddle_linear_regression.py
import pandas as pd
import numpy as np

import bentoml
from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import DataframeInput
from bentoml.frameworks.paddle import PaddlePaddleModelArtifact

@env(infer_pip_packages=True)
@artifacts([PaddlePaddleModelArtifact('model')])
class PaddleLinearRegression(bentoml.BentoService):

  @api(input=DataframeInput(), batch=True)
  def predict(self, df: pd.DataFrame):
        input_data = df.to_numpy().astype('float32')

        predictor = self.artifacts.model
        input_names = predictor.get_input_names()
        input_handle = predictor.get_input_handle(input_names[0])

        input_handle.reshape(input_data.shape)
        input_handle.copy_from_cpu(input_data)

        predictor.run()

        output_names = predictor.get_output_names()
        output_handle = predictor.get_output_handle(output_names[0])
        output_data = output_handle.copy_to_cpu()

        return output_data

Writing paddle_linear_regression.py


In [12]:
# 1) import the custom BentoService defined above
from paddle_linear_regression import PaddleLinearRegression

# 2) `pack` it with required artifacts
bento_svc = PaddleLinearRegression()
bento_svc.pack('model', model)

# 3) save your BentoSerivce
saved_path = bento_svc.save()


[2021-04-04 19:01:27,215] WARNING - Using BentoML not from official PyPI release. In order to find the same version of BentoML when deploying your BentoService, you must set the 'core/bentoml_deploy_version' config to a http/git location of your BentoML fork, e.g.: 'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'


2021-04-04 19:01:28,269 - INFO - Context impl SQLiteImpl.
2021-04-04 19:01:28,270 - INFO - Will assume non-transactional DDL.


[2021-04-04 19:01:28,740] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.12.0, but loading from BentoML version 0.12.0+15.g7d40998
[2021-04-04 19:01:28,779] INFO - BentoService bundle 'PaddleLinearRegression:20210404190128_F4577F' saved to: /Users/bozhaoyu/bentoml/repository/PaddleLinearRegression/20210404190128_F4577F


# **REST API Model Serving**

In [ ]:
!bentoml serve PaddleLinearRegression:latest

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
[2021-04-01 21:29:16,663] INFO - Getting latest version PaddleLinearRegression:20210401212846_F88F15
[2021-04-01 21:29:16,683] INFO - Starting BentoML API proxy in development mode..
[2021-04-01 21:29:16,685] INFO - Starting BentoML API server in development mode..
[2021-04-01 21:29:16,890] WARNING - Using BentoML not from official PyPI release. In order to find the same version of BentoML when deploying your BentoService, you must set the 'core/bentoml_deploy_version' config to a http/git location of your BentoML fork, e.g.: 'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2021-04-01 21:29:16,911] WARNING - Saved BentoService bundle 

If you are running this notebook from Google Colab, you can start the dev server with --run-with-ngrok option, to gain acccess to the API endpoint via a public endpoint managed by ngrok:

In [ ]:
!bentoml serve PaddleLinearRegression:latest --run-with-ngrok

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
[2021-04-01 21:30:36,588] INFO - Getting latest version PaddleLinearRegression:20210401212846_F88F15
[2021-04-01 21:30:36,609] INFO - Starting BentoML API proxy in development mode..
[2021-04-01 21:30:36,611] INFO - Starting BentoML API server in development mode..
[2021-04-01 21:30:36,731] WARNING - Using BentoML not from official PyPI release. In order to find the same version of BentoML when deploying your BentoService, you must set the 'core/bentoml_deploy_version' config to a http/git location of your BentoML fork, e.g.: 'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2021-04-01 21:30:36,750] WARNING - Saved BentoService bundle 

# **Make request to the REST server**

*After navigating to the location of this notebook, copy and paste the following code to your terminal and run it to make request*

In [ ]:
curl -i \
--request POST \
--header "Content-Type: text/csv" \
-d @test.csv \
localhost:5000/predict

# **Containerize model server with Docker**

One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **not available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the PaddleLinearRegression prediction service created above:

In [ ]:
!bentoml containerize PaddleLinearRegression:latest

In [ ]:
!docker run --rm -p 5000:5000 PaddleLinearRegression:20210306050051_766D0A

/bin/bash: docker: command not found


# **Load Saved Bento Service**

In [ ]:
#TODO ADD INPUT
from bentoml import load

svc = load(saved_path)

input = pd.DataFrame([[-0.0405441 ,  0.06636364, -0.32356227, -0.06916996, -0.03435197,
        0.05563625, -0.03475696,  0.02682186, -0.37171335, -0.21419304,
       -0.33569506,  0.10143217, -0.21172912]]).astype('float32')

print(svc.predict(input))

[2021-03-06 05:05:25,666] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.7.8, but loading from BentoML version 0.7.8+402.g8c47823
[2021-03-06 05:05:25,669] WARNING - Module `paddle_linear_regression` already loaded, using existing imported module.
[2021-03-06 05:05:25,680] WARNING - pip package requirement pandas already exist
[2021-03-06 05:05:25,681] WARNING - pip package requirement paddlepaddle already exist
[[0.85793805]]


# **Launch inference job from CLI**

In [ ]:
!bentoml run PaddleLinearRegression:latest predict --format csv --input-file test.csv

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
[2021-04-01 21:29:58,275] INFO - Getting latest version PaddleLinearRegression:20210401212846_F88F15
[2021-04-01 21:29:58,962] WARNING - Using BentoML not from official PyPI release. In order to find the same version of BentoML when deploying your BentoService, you must set the 'core/bentoml_deploy_version' config to a http/git location of your BentoML fork, e.g.: 'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2021-04-01 21:29:58,978] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.12.0, but loading from BentoML version 0.12.0+14.g0fac537
/usr/local/lib/python3.7/dist-p

# **Deployment Options**

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:

* [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
* [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
* [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:

* [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
* [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
* [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
* [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:

* [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
* [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
* [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
* [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
* [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)